In [6]:
from head_detector import HeadDetector
import cv2
import os

# Inicializar el detector
detector = HeadDetector()

# Directorio de entrada (donde están las imágenes originales)
input_dir = "/home/jocareher/Downloads/baby_face_72/images"

# Directorio de salida (donde se guardarán las imágenes procesadas)
output_dir = "/home/jocareher/Downloads/baby_face_72/results"

# Crear el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Iterar sobre todos los archivos en el directorio de entrada
for filename in os.listdir(input_dir):
    # Construir la ruta completa del archivo
    input_path = os.path.join(input_dir, filename)
    
    # Verificar que sea un archivo de imagen (puedes ajustar los formatos permitidos)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Obtener las predicciones
        predictions = detector(input_path)
        
        # Dibujar los resultados en la imagen
        result_image = predictions.draw()
        image_rgb = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        
        # Convertir de BGR a RGB y guardar la imagen procesada
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, image_rgb)

print(f"Imágenes procesadas guardadas en: {output_dir}")


Imágenes procesadas guardadas en: /home/jocareher/Downloads/baby_face_72/results


In [7]:
import cv2
import os
import numpy as np
from head_detector import HeadDetector

# Inicializar el detector
detector = HeadDetector()

# Índices de los 68 landmarks habituales (ajusta según corresponda a tu modelo)
LANDMARK_INDICES_68 = [
    # Añade aquí los índices que correspondan a los landmarks habituales
    # Por ejemplo:
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,  # Contorno de la mandíbula
    17, 18, 19, 20, 21, 22, 23, 24, 25, 26,                    # Cejas
    27, 28, 29, 30,                                           # Nariz superior
    31, 32, 33, 34, 35,                                       # Nariz inferior
    36, 37, 38, 39, 40, 41,                                   # Ojo derecho
    42, 43, 44, 45, 46, 47,                                   # Ojo izquierdo
    48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67
]  # Boca

# Directorio de entrada y salida
input_dir = "/home/jocareher/Downloads/baby_face_72/images"
output_dir = "/home/jocareher/Downloads/results_landmarks"
os.makedirs(output_dir, exist_ok=True)

def draw_68_landmarks(image, landmarks):
    """Dibuja los 68 landmarks en la imagen."""
    for idx in LANDMARK_INDICES_68:
        x, y = int(landmarks[idx][0]), int(landmarks[idx][1])
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
    return image

# Procesar todas las imágenes
for filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, filename)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Obtener predicciones
        predictions = detector(input_path)

        # Extraer los landmarks en 2D
        all_landmarks = predictions.vertices_3d[:, :2]

        # Filtrar los landmarks a los 68 habituales
        landmarks_68 = np.take(all_landmarks, LANDMARK_INDICES_68, axis=0)

        # Cargar la imagen original
        image = cv2.imread(input_path)

        # Dibujar los 68 landmarks
        image_with_landmarks = draw_68_landmarks(image, landmarks_68)

        # Guardar la imagen procesada
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, image_with_landmarks)

print(f"Imágenes con 68 landmarks guardadas en: {output_dir}")


AttributeError: 'PredictionResult' object has no attribute 'vertices_3d'

In [2]:
from head_detector import HeadDetector
import cv2
detector = HeadDetector()
image_path = "/home/jocareher/Downloads/DSLPT/test.jpg"
predictions = detector(image_path)
# predictions.heads contain a list of heads with .bbox, .vertices_3d, .head_pose params
result_image = predictions.draw() # draw heads on the image
cv2.imwrite("result.png",cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)) # save result image to preview it.

True

In [7]:
import numpy as np

def load_landmarks(file_path: str) -> np.ndarray:
    """
    Carga los landmarks de un archivo en formato x1 y1 x2 y2 ... xn yn.
    :param file_path: Ruta del archivo.
    :return: Matriz de coordenadas de forma (n, 2).
    """
    with open(file_path, "r") as file:
        lines = file.readlines()
    # Convertir cada línea en una matriz de coordenadas 2D
    landmarks = [np.array(line.strip().split(), dtype=int).reshape(-1, 2) for line in lines]
    return np.vstack(landmarks)  # Si es un solo set, devuelve la matriz 2D

def match_unique_landmarks(dlib_landmarks: np.ndarray, vgg_landmarks: np.ndarray, coord_output_file: str, index_output_file: str):
    """
    Guarda los puntos más cercanos en formato x1 y1 ... xn yn para las coordenadas y indices = [indx] para los índices.

    :param dlib_landmarks: np.ndarray de forma (68, 2), coordenadas de DLIB.
    :param vgg_landmarks: np.ndarray de forma (N, 2), coordenadas de VGG Heads.
    :param coord_output_file: Nombre del archivo donde se guardarán las coordenadas en una línea.
    :param index_output_file: Nombre del archivo donde se guardarán los índices en formato de lista.
    """
    matched_indices = []
    matched_landmarks = []
    used_indices = set()

    for dlib_point in dlib_landmarks[:68]:  # Asegurarse de procesar solo 68 puntos
        distances = np.linalg.norm(vgg_landmarks - dlib_point, axis=1)
        for idx in used_indices:
            distances[idx] = np.inf
        closest_index = np.argmin(distances)
        used_indices.add(closest_index)
        matched_indices.append(closest_index)
        matched_landmarks.append(vgg_landmarks[closest_index])

    # Guardar las coordenadas en una línea
    with open(coord_output_file, "w") as coord_file:
        flattened_landmarks = [f"{int(coord[0])} {int(coord[1])}" for coord in matched_landmarks]
        coord_file.write(" ".join(flattened_landmarks) + "\n")

    # Guardar los índices en formato de lista
    with open(index_output_file, "w") as index_file:
        index_file.write(f"indices = {matched_indices}\n")

    print(f"Coordenadas guardadas en {coord_output_file}")
    print(f"Índices guardados en {index_output_file}")

# Cargar los archivos de entrada
dlib_landmarks = np.loadtxt("test.txt").reshape(-1, 2)  # Archivo con 68 landmarks
vgg_landmarks = np.loadtxt("landmarks_2d.txt").reshape(-1, 2)  # Archivo con 2500+ landmarks

# Ejecutar el cálculo
match_unique_landmarks(dlib_landmarks, vgg_landmarks, "matched_coordinates.txt", "matched_indices.txt")


Coordenadas guardadas en matched_coordinates.txt
Índices guardados en matched_indices.txt


In [7]:
import os
from head_detector import HeadDetector

# Inicializar el detector
detector = HeadDetector()

# Directorio de entrada
input_dir = "/home/jocareher/Downloads/baby_face_72/images"

# Directorio de salida
output_dir = "/home/jocareher/Downloads/baby_face_72/landmarks_txt"
os.makedirs(output_dir, exist_ok=True)

# Iterar sobre los archivos en el directorio
for filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, filename)
    
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Obtener predicciones y procesar
        predictions = detector(input_path)
        
        # Crear archivo .txt para la imagen actual
        output_txt_path = os.path.join(output_dir, os.path.splitext(filename)[0] + ".txt")
        with open(output_txt_path, "w") as file:
            for head in predictions.heads:
                vertices = head.vertices_3d  # Coordenadas de los landmarks procesadas dentro de _parse_predictions
                flattened = vertices.flatten().astype(int)  # Asegurarse de que sean enteros
                file.write(" ".join(map(str, flattened)) + "\n")

print(f"Archivos .txt guardados en: {output_dir}")




Archivos .txt guardados en: /home/jocareher/Downloads/baby_face_72/landmarks_txt
